In [26]:
import pyspark.sql.functions as f
from pyspark.sql.functions import col

StatementMeta(hackatchonpool, 5, 27, Finished, Available)

# Get dataframes from bronze

In [8]:
bronze_location = "abfss://bronze@synapsehackst.dfs.core.windows.net/"
observationSchema = spark.read.json(bronze_location+"historic_data/observation/year=2016/part-00000-190423bd-20a1-40a2-910f-3388f2d78a2c.c000.json").schema
observation_df = spark.read.schema(observationSchema).json(bronze_location+"historic_data/observation/year=2016/part-00000-190423bd-20a1-40a2-910f-3388f2d78a2c.c000.json")
claimSchema = spark.read.json(bronze_location+"historic_data/claim/year=2016/part-00000-3568a0cf-2179-4481-b6dd-1b210f01ba9f.c000.json").schema
claim_df = spark.read.schema(claimSchema).json(bronze_location+"historic_data/claim/year=2016/part-00000-3568a0cf-2179-4481-b6dd-1b210f01ba9f.c000.json")

StatementMeta(hackatchonpool, 5, 9, Finished, Available)

In [9]:
observation_df.printSchema()

StatementMeta(hackatchonpool, 5, 10, Finished, Available)

root
 |-- category: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coding: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |-- display: string (nullable = true)
 |    |    |    |    |-- system: string (nullable = true)
 |-- code: struct (nullable = true)
 |    |-- coding: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |-- text: string (nullable = true)
 |-- component: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- code: struct (nullable = true)
 |    |    |    |-- coding: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- code: string (nullabl

In [10]:
obs_category_df=observation_df.withColumn("cat1", f.explode('category'))\
    .select("cat1.*")\
    .withColumn("cat2", f.explode('coding'))\
    .select("cat2.*")

StatementMeta(hackatchonpool, 5, 11, Finished, Available)

In [11]:
obs_code_df=observation_df\
    .select("code.*")\
    .withColumn("cat2", f.explode('coding'))

StatementMeta(hackatchonpool, 5, 12, Finished, Available)

In [35]:
obs_component_df=observation_df.withColumn("cat1", f.explode('component'))\
    .select("cat1.*")\
    .select("code.*", "valueQuantity.*")\
    .withColumn("cat2", f.explode('coding'))\
    .select("text", "code", "system", "unit", "value", col("cat2.code").alias("cat2code"), col("cat2.display"), col("cat2.system").alias("cat2system"))

StatementMeta(hackatchonpool, 5, 36, Finished, Available)

In [13]:
claim_df.printSchema()

StatementMeta(hackatchonpool, 5, 14, Finished, Available)

root
 |-- billablePeriod: struct (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- start: string (nullable = true)
 |-- created: string (nullable = true)
 |-- diagnosis: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- diagnosisReference: struct (nullable = true)
 |    |    |    |-- reference: string (nullable = true)
 |    |    |-- sequence: long (nullable = true)
 |-- id: string (nullable = true)
 |-- insurance: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coverage: struct (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |-- focal: boolean (nullable = true)
 |    |    |-- sequence: long (nullable = true)
 |-- item: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- diagnosisSequence: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- encounter: array (nullable = true)
 |    |

In [14]:
claim_diagnosis_df=claim_df.withColumn("cat1", f.explode('diagnosis'))\
    .select("cat1.*")\
    .select("diagnosisReference", "sequence")

StatementMeta(hackatchonpool, 5, 15, Finished, Available)

In [15]:
claim_insurance_df=claim_df.withColumn("cat1", f.explode('insurance'))\
    .select("cat1.*")

StatementMeta(hackatchonpool, 5, 16, Finished, Available)

In [16]:
claim_procedure_df=claim_df.withColumn("cat1", f.explode('procedure'))\
    .select("cat1.*")

StatementMeta(hackatchonpool, 5, 17, Finished, Available)

# Write to silver

In [18]:
silver_location = "abfss://silver@synapsehackst.dfs.core.windows.net/"
write_mode="overwrite"

StatementMeta(hackatchonpool, 5, 19, Finished, Available)

In [ ]:
claim_df.coalesce(200).write.format("delta").option("path", silver_location+"claim_main_").mode(write_mode).saveAsTable("fhirdb.claim_main")
claim_diagnosis_df.coalesce(200).write.format("delta").option("path", silver_location+"claim_diagnosis").mode(write_mode).saveAsTable("fhirdb.claim_diagnosis")
claim_insurance_df.coalesce(200).write.format("delta").option("path", silver_location+"claim_insurance").mode(write_mode).saveAsTable("fhirdb.claim_insurance")
claim_procedure_df.coalesce(200).write.format("delta").option("path", silver_location+"claim_procedure").mode(write_mode).saveAsTable("fhirdb.claim_procedure")

In [37]:
observation_df.coalesce(200).write.format("delta").option("path", silver_location+"observation").mode(write_mode).saveAsTable("fhirdb.observation_main")
obs_category_df.coalesce(200).write.format("delta").option("path", silver_location+"category").mode(write_mode).saveAsTable("fhirdb.category")
obs_code_df.coalesce(200).write.format("delta").option("path", silver_location+"code").mode(write_mode).saveAsTable("fhirdb.code")
obs_component_df.coalesce(200).write.format("delta").option("path", silver_location+"component").mode(write_mode).saveAsTable("fhirdb.component")

StatementMeta(hackatchonpool, 5, 38, Finished, Available)